In [2]:
import pandas as pd
from Bio import SeqIO

# --- 1. 定义文件路径 ---
synteny_excel_file = "共线表_41588_2023_1620_MOESM2_ESM.xlsx"
synteny_sheet_name = "Supplementary Table 2a"
homology_excel_file = "同源表_41588_2023_1620_MOESM4_ESM.xlsx"
homology_sheet_name = "human vs mouse"

human_fasta_file = "gencode.v25.lncRNA_transcripts.fa"
mouse_fasta_file = "gencode.vM10.lncRNA_transcripts.fa"

output_human_fasta = "human_syntenic_not_homology_lncRNAs.fa"
output_mouse_fasta = "mouse_syntenic_not_homology_lncRNAs.fa"
output_summary_file = "syntenic_not_homology_summary.txt"

# --- 2. 读取共线对Excel文件 ---
print("正在读取共线表Excel文件...")
synteny_df = pd.read_excel(synteny_excel_file, sheet_name=synteny_sheet_name)

# 提取人类和小鼠基因ID对（去除版本号）
synteny_pairs = set()
for index, row in synteny_df.iterrows():
    human_gene = str(row[synteny_df.columns[0]]).strip()
    mouse_gene = str(row[synteny_df.columns[1]]).strip()
    
    if human_gene != 'nan' and mouse_gene != 'nan' and human_gene and mouse_gene:
        # 去除版本号
        if '.' in human_gene:
            human_gene = human_gene.split('.')[0]
        if '.' in mouse_gene:
            mouse_gene = mouse_gene.split('.')[0]
        
        synteny_pairs.add((human_gene, mouse_gene))

print(f"共读取 {len(synteny_pairs)} 对共线基因对。")

# --- 3. 读取同源对Excel文件 ---
print("正在读取同源表Excel文件...")
homology_df = pd.read_excel(homology_excel_file, sheet_name=homology_sheet_name)

# 提取人类和小鼠基因ID对（去除版本号）
homology_pairs = set()
for index, row in homology_df.iterrows():
    human_gene = str(row[homology_df.columns[0]]).strip()
    mouse_gene = str(row[homology_df.columns[1]]).strip()
    
    if human_gene != 'nan' and mouse_gene != 'nan' and human_gene and mouse_gene:
        # 去除版本号
        if '.' in human_gene:
            human_gene = human_gene.split('.')[0]
        if '.' in mouse_gene:
            mouse_gene = mouse_gene.split('.')[0]
        
        homology_pairs.add((human_gene, mouse_gene))

print(f"共读取 {len(homology_pairs)} 对同源基因对。")

# --- 4. 找出共线但不同源的基因对 ---
syntenic_not_homology_pairs = synteny_pairs - homology_pairs
print(f"找到 {len(syntenic_not_homology_pairs)} 对共线但不同源的基因对。")

# --- 5. 构建基因到最长转录本的映射 ---
def build_gene_to_longest_transcript(fasta_file):
    """构建基因ID到最长转录本ID的映射"""
    gene_to_transcript = {}
    
    print(f"正在分析 {fasta_file} 中的转录本...")
    
    for record in SeqIO.parse(fasta_file, "fasta"):
        header = record.id
        seq_length = len(record.seq)
        
        # 从header中提取基因ID（去除版本号）
        parts = header.split('|')
        if len(parts) >= 2:
            gene_id_with_version = parts[1]
            gene_id = gene_id_with_version.split('.')[0]  # 去除版本号
            
            # 如果基因还没有映射，或者当前转录本更长，则更新映射
            if gene_id not in gene_to_transcript or seq_length > gene_to_transcript[gene_id][1]:
                gene_to_transcript[gene_id] = (header, seq_length)  # 存储完整ID和长度
    
    print(f"为 {len(gene_to_transcript)} 个基因找到了最长转录本")
    return gene_to_transcript

# 为人类和小鼠分别构建映射
human_gene_to_transcript = build_gene_to_longest_transcript(human_fasta_file)
mouse_gene_to_transcript = build_gene_to_longest_transcript(mouse_fasta_file)

# --- 6. 索引FASTA文件 ---
print("正在索引人类FASTA文件...")
human_idx = SeqIO.index(human_fasta_file, "fasta")
print("正在索引小鼠FASTA文件...")
mouse_idx = SeqIO.index(mouse_fasta_file, "fasta")

# --- 7. 提取共线但不同源的序列 ---
def write_single_line_fasta(record, output_handle):
    """将序列写入为单行FASTA格式"""
    output_handle.write(f">{record.id}\n")
    output_handle.write(f"{str(record.seq)}\n")

# 创建汇总信息
summary_lines = []

with open(output_human_fasta, 'w') as human_out, open(output_mouse_fasta, 'w') as mouse_out:
    human_count = 0
    mouse_count = 0
    processed_pairs = 0
    found_pairs = 0

    # 遍历每一对共线但不同源的基因
    for human_gene, mouse_gene in syntenic_not_homology_pairs:
        found_human = False
        found_mouse = False
        
        # 获取人类基因的最长转录本
        if human_gene in human_gene_to_transcript:
            transcript_id = human_gene_to_transcript[human_gene][0]
            if transcript_id in human_idx:
                seq_record = human_idx[transcript_id]
                write_single_line_fasta(seq_record, human_out)
                found_human = True
                human_count += 1
        
        # 获取小鼠基因的最长转录本
        if mouse_gene in mouse_gene_to_transcript:
            transcript_id = mouse_gene_to_transcript[mouse_gene][0]
            if transcript_id in mouse_idx:
                seq_record = mouse_idx[transcript_id]
                write_single_line_fasta(seq_record, mouse_out)
                found_mouse = True
                mouse_count += 1
        
        # 记录汇总信息
        status = "成功" if found_human and found_mouse else "部分成功" if found_human or found_mouse else "失败"
        summary_lines.append(f"{human_gene}\t{mouse_gene}\t{status}")
        
        # 统计找到的配对
        if found_human and found_mouse:
            found_pairs += 1
        
        processed_pairs += 1
        if processed_pairs % 1000 == 0:
            print(f"已处理 {processed_pairs}/{len(syntenic_not_homology_pairs)} 对基因...")

print("\n完成！")
print(f"从人类文件中提取了 {human_count} 条序列到 {output_human_fasta}")
print(f"从小鼠文件中提取了 {mouse_count} 条序列到 {output_mouse_fasta}")
print(f"成功找到 {found_pairs}/{len(syntenic_not_homology_pairs)} 对完全匹配的基因")

# 保存汇总信息
with open(output_summary_file, 'w') as summary_out:
    summary_out.write("人类基因\t小鼠基因\t提取状态\n")
    for line in summary_lines:
        summary_out.write(f"{line}\n")

# 关闭索引以释放资源
human_idx.close()
mouse_idx.close()

# 显示一些统计信息
print("\n=== 统计信息 ===")
print(f"总共线对: {len(synteny_pairs)}")
print(f"总同源对: {len(homology_pairs)}")
print(f"共线但不同源对: {len(syntenic_not_homology_pairs)}")
print(f"成功提取的人类序列: {human_count}")
print(f"成功提取的小鼠序列: {mouse_count}")
print(f"汇总信息已保存到: {output_summary_file}")

正在读取共线表Excel文件...
共读取 25005 对共线基因对。
正在读取同源表Excel文件...
共读取 7355 对同源基因对。
找到 17653 对共线但不同源的基因对。
正在分析 gencode.v25.lncRNA_transcripts.fa 中的转录本...
为 15755 个基因找到了最长转录本
正在分析 gencode.vM10.lncRNA_transcripts.fa 中的转录本...
为 9856 个基因找到了最长转录本
正在索引人类FASTA文件...
正在索引小鼠FASTA文件...
已处理 1000/17653 对基因...
已处理 2000/17653 对基因...
已处理 3000/17653 对基因...
已处理 4000/17653 对基因...
已处理 5000/17653 对基因...
已处理 6000/17653 对基因...
已处理 7000/17653 对基因...
已处理 8000/17653 对基因...
已处理 9000/17653 对基因...
已处理 10000/17653 对基因...
已处理 11000/17653 对基因...
已处理 12000/17653 对基因...
已处理 13000/17653 对基因...
已处理 14000/17653 对基因...
已处理 15000/17653 对基因...
已处理 16000/17653 对基因...
已处理 17000/17653 对基因...

完成！
从人类文件中提取了 17653 条序列到 human_syntenic_not_homology_lncRNAs.fa
从小鼠文件中提取了 17653 条序列到 mouse_syntenic_not_homology_lncRNAs.fa
成功找到 17653/17653 对完全匹配的基因

=== 统计信息 ===
总共线对: 25005
总同源对: 7355
共线但不同源对: 17653
成功提取的人类序列: 17653
成功提取的小鼠序列: 17653
汇总信息已保存到: syntenic_not_homology_summary.txt


In [12]:
# 在现有代码的基础上添加验证部分

# 找出共线对和同源对的交集
synteny_homology_intersection = synteny_pairs & homology_pairs
print(f"共线对和同源对的交集大小: {len(synteny_homology_intersection)}")

# 验证集合运算
print(f"共线对大小: {len(synteny_pairs)}")
print(f"同源对大小: {len(homology_pairs)}")
print(f"共线但不同源对: {len(syntenic_not_homology_pairs)}")
print(f"共线对与同源对交集大小: {len(synteny_homology_intersection)}")
print(f"共线对与同源对并集大小: {len(synteny_pairs | homology_pairs)}")
print(f"共线对与同源对差集大小: {len(synteny_pairs - homology_pairs)}")

# 检查是否有重复项
print(f"共线对中是否有重复: {len(synteny_df) != len(synteny_pairs)}")
print(f"同源对中是否有重复: {len(homology_df) != len(homology_pairs)}")

# 找出只在同源表中出现但不在共线表中的基因对
homology_only_pairs = homology_pairs - synteny_pairs
print(f"只在同源表中出现的基因对数量: {len(homology_only_pairs)}")

if homology_only_pairs:
    print("只在同源表中出现的基因对:")
    for i, pair in enumerate(list(homology_only_pairs)):
        print(f"  {i+1}. {pair}")

共线对和同源对的交集大小: 7352
共线对大小: 25005
同源对大小: 7355
共线但不同源对: 17653
共线对与同源对交集大小: 7352
共线对与同源对并集大小: 25008
共线对与同源对差集大小: 17653
共线对中是否有重复: False
同源对中是否有重复: False
只在同源表中出现的基因对数量: 3
只在同源表中出现的基因对:
  1. ('ENSG00000250954', 'ENSMUSG00000106044')
  2. ('ENSG00000230768', 'ENSMUSG00000105882')
  3. ('ENSG00000224944', 'ENSMUSG00000085522')
